# Find reasonable parameter scales from Sage

We will scale the parameters to 1 / \<mean value in Sage>.

In [1]:
from openff.toolkit import ForceField
from openff.units import unit
from functools import partial
import numpy as np

PARAMETER_COLS = {
    "Bonds": ["k", "length"],
    "Angles": ["k", "angle"],
    "ProperTorsions": ["k"],
    "ImproperTorsions": ["k"]
}

LINEAR_BASIS_FNS = {
    "Bonds": [lambda x: x - 0.4 * unit.angstrom, lambda x: x + 0.4 * unit.angstrom],
    # "Angles": [lambda x: 0.0, lambda x: np.pi]
    # "Angles": [lambda x: x * 0.5, lambda x: x * 1.5]
    "Angles": [lambda x: max(0.0, x - np.pi/3 * unit.radian), lambda x: min(np.pi * unit.radian, x + np.pi/3 * unit.radian)]
}

def get_parameter_col_values(ff: ForceField, parameter_type: str, parameter_cols: list[str]) -> list[dict[str, float]]:
    values = []
    for parameter in ff.get_parameter_handler(parameter_type).parameters:
        vals = {}
        for parameter_col in parameter_cols:
            val = getattr(parameter, parameter_col)
            if val is not None:
                vals[parameter_col] = val
        values.append(vals)
    return values

def _get_mean(values: list[float]) -> float:
    if not isinstance(values, (list, tuple)):
        return values
    return sum(values) / len(values)

def get_reciprocal_of_mean_of_dicts(values: list[dict[str, float]]) -> dict[str, float]:
    mean_values = {}
    for parameter_col in values[0].keys():
        col_values = [_get_mean(v[parameter_col]) for v in values if parameter_col in v]
        mean_values[parameter_col] =  1 /  _get_mean(col_values)
    return mean_values

def linearise_pair(force_constant: float, equil_val: float, basis_fns: list) -> list[float]:
    equil_val_linear_lower = basis_fns[0](equil_val)
    equil_val_linear_upper = basis_fns[1](equil_val)
    k1 = force_constant * (equil_val_linear_upper - equil_val) / (equil_val_linear_upper - equil_val_linear_lower)
    k2 = force_constant * (equil_val - equil_val_linear_lower) / (equil_val_linear_upper - equil_val_linear_lower)
    return {"k1": k1, "k2": k2}

ff = ForceField("openff-2.2.1.offxml")

In [2]:
for parameter_type, parameter_cols in PARAMETER_COLS.items():
    print(f"Processing parameter type: {parameter_type}")
    values = get_parameter_col_values(ff, parameter_type, parameter_cols)
    if parameter_type in LINEAR_BASIS_FNS:
        converted_values = []
        basis_fns = LINEAR_BASIS_FNS[parameter_type]
        linearise_fn = partial(linearise_pair, basis_fns=basis_fns)
        values = [linearise_fn(v["k"], v[parameter_cols[1]]) for v in values]
    mean_values = get_reciprocal_of_mean_of_dicts(values)
    print(f"Mean values for {parameter_type}: {mean_values}\n")


Processing parameter type: Bonds
Mean values for Bonds: {'k1': <Quantity(0.00284063965, 'angstrom ** 2 / kilocalorie_per_mole')>, 'k2': <Quantity(0.00284063965, 'angstrom ** 2 / kilocalorie_per_mole')>}

Processing parameter type: Angles
Mean values for Angles: {'k1': <Quantity(0.0124776193, 'radian ** 2 / kilocalorie_per_mole')>, 'k2': <Quantity(0.0108682968, 'radian ** 2 / kilocalorie_per_mole')>}

Processing parameter type: ProperTorsions
Mean values for ProperTorsions: {'k': <Quantity(1.33206984, 'mole / kilocalorie')>}

Processing parameter type: ImproperTorsions
Mean values for ImproperTorsions: {'k': <Quantity(0.122777484, 'mole / kilocalorie')>}

